In [1]:
import sys
sys.path.append('C:/Users/32827/Documents/GitHub/BackdoorBox/')
import core
# import LIRA

C:\Users\32827\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import math
from tqdm import trange
import numpy as np
import os, sys
import pdb
from PIL import Image
from numpy import asarray
from skimage.transform import resize
import random
from art.estimators.classification import PyTorchClassifier
from art.utils import load_dataset
from torchvision.models.resnet import BasicBlock, Bottleneck
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torch.nn.functional as F
from torchvision.datasets import DatasetFolder, CIFAR10

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss,Softmax
from torch.optim import LBFGS

from qiskit import QuantumCircuit
from qiskit.utils import algorithm_globals
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F
from torchvision.transforms import Compose, ToTensor, PILToTensor, RandomHorizontalFlip

In [4]:
# Prepare datasets and follow the default data augmentation in the original paper
transform_train = Compose([
    transforms.Resize((32, 32)),
    ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                        (0.5, 0.5, 0.5))
])
transform_test = Compose([
    transforms.Resize((32, 32)),
    ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                        (0.5, 0.5, 0.5))
])


In [5]:
trainset = CIFAR10(
    root='../data', # please replace this with path to your dataset
    transform=transform_train,
    target_transform=None,
    train=True,
    download=True)

Files already downloaded and verified


In [6]:
idx = []
trainset.targets = np.array(trainset.targets).astype('int64')
for targets in range(4):
    classes = np.where(trainset.targets == targets)[0]
    print(classes)
    idx = np.append(idx,random.choices(classes,k = 1000))
idx = idx.astype(int)
print(idx)
trainset.data = trainset.data[idx]
trainset.targets = trainset.targets[idx]

[   29    30    35 ... 49941 49992 49994]
[    4     5    32 ... 49993 49998 49999]
[    6    13    18 ... 49987 49991 49995]
[    9    17    21 ... 49979 49982 49983]
[49921 38980 45102 ... 22793 33978 33275]


In [7]:
testset = CIFAR10(
    root='../data', # please replace this with path to your dataset
    transform=transform_test,
    target_transform=None,
    train=False,
    download=True)

Files already downloaded and verified


In [8]:
idx = []
testset.targets = np.array(testset.targets).astype('int64')
for targets in range(4):
    classes = np.where(testset.targets == targets)[0]
    # print(classes)
    idx = np.append(idx,random.choices(classes,k = 100))
idx = idx.astype(int)
# print(idx)
testset.data = testset.data[idx]
testset.targets = testset.targets[idx]

In [9]:
def create_qnn():
    feature_map = ZZFeatureMap(3)
    ansatz = RealAmplitudes(3, reps=1)
    qc = QuantumCircuit(3)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)
    
    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    parity = lambda x: "{:b}".format(x).count("1") % 4  # optional interpret function
    output_shape = 4  # parity = 0, 1
    qnn = SamplerQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
        interpret=parity,
        output_shape=output_shape,
    )
    return qnn


qnn4 = create_qnn()

In [10]:
# Define torch NN module


class Net(Module):
    def __init__(self,qnn):
        super().__init__()
        self.conv1 = Conv2d(3, 6, 5)
        self.pool = MaxPool2d(2, 2)
        self.conv2 = Conv2d(6, 16, 5)
        self.fc1 = Linear(16 * 5 * 5, 120)
        self.fc2 = Linear(120, 84)
        # self.qnn = TorchConnector(qnn)  # Apply torch connector, weights chosen
        # uniformly at random from interval [-1,1].
        self.fc3 = Linear(84,4)  # 1-dimensional output from QNN
        # self.qnn = TorchConnector(qnn)  # Apply torch connector, weights chosen
        self.fc4 = Linear(4,4)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # x = self.qnn(x)  # apply QNN
        x = self.fc4(x)
        return x

classifier_model = Net(qnn4)

In [16]:
schedule = {
    'device': 'GPU',
    'CUDA_VISIBLE_DEVICES': '1',
    'GPU_num': 1,

    'benign_training': False,
    'batch_size': 4,
    'num_workers': 8,

    'lr': 0.0001,
    'lr_atk': 0.0001,
    'momentum': 0.9,
    
    'epochs': 10,
    'train_epoch': 1,
    'cls_test_epoch': 5,


    'tune_test_epochs': 1,
    'tune_test_lr': 0.0001,
    'tune_momentum': 0.9,
    'tune_weight_decay': 5e-4,
    'tune_test_epoch_interval': 1,

    'schedulerC_lambda': 0.1,
    'schedulerC_milestones': '50,100,150,200',

    'log_iteration_interval': 100,
    'test_epoch_interval': 10,
    'save_epoch_interval': 10,

    'pretrain':"./models/normal-cifar10_mdl.pt",
    'save_dir': 'experiments',
    'experiment_name': 'train_poison_DataFolder_CIFAR10_LIRA'
    # 'num_workers' : 1
}

In [17]:
LIRA = core.LIRA(
    dataset_name="ncifar10",
    train_dataset=trainset,
    test_dataset=testset,
    model=classifier_model, #core.models.vgg11(num_classes=10), #core.models.ResNet(18),
    loss=nn.CrossEntropyLoss(),
    y_target=1,
    eps=0.001,
    alpha=0.5,
    tune_test_eps=0.0001,
    tune_test_alpha=0.5,
    best_threshold=0.1,
    schedule=schedule,
)

In [18]:
torch.cuda.current_device()

0

In [19]:
torch.cuda.device_count()

1

In [20]:
# Train backdoored model
LIRA.train()

# Get the poisoned dataset
poisoned_train_dataset, poisoned_test_dataset = LIRA.get_poisoned_dataset()

Pretrain activated
This machine has 1 cuda devices, and use 1 of them to train.
Total train samples: 4000
Total test samples: 400
Batch size: 4
iteration every epoch: 1000
Initial learning rate: 0.0001

[2024-03-11_21:02:05] Train [1] Loss: clean 1.3982 poison 7.3508 total 4.3745 Tri 1.4261

==========Test result on benign test dataset==========
[2024-03-11_21:02:12] Top-1 correct / Total: 157/400, Top-1 accuracy: 0.3925, Top-3 correct / Total: 157/400, Top-3 accuracy: 0.3925 time: 59.01947617530823

==========Test result on poisoned test dataset==========
[2024-03-11_21:02:12] Top-1 correct / Total: 332/400, Top-1 accuracy: 0.83, Top-3 correct / Total: 332/400, Top-3 accuracy: 0.83, time: 59.01947617530823

[2024-03-11_21:02:31] Train [2] Loss: clean 0.8842 poison 4.2086 total 2.5464 Tri 1.4259

==========Test result on benign test dataset==========
[2024-03-11_21:02:35] Top-1 correct / Total: 183/400, Top-1 accuracy: 0.4575, Top-3 correct / Total: 183/400, Top-3 accuracy: 0.4575 time